$\newcommand{\To}{\Rightarrow}$
$\newcommand{\false}{\mathrm{false}}$

In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import TFun, BoolType, NatType
from kernel.term import Var, Forall, Exists, Nat, And, Inst, Lambda, Eq
from kernel.thm import Thm
from kernel.proofterm import ProofTerm
from kernel import theory
from logic import basic
from logic.logic import apply_theorem
from syntax.settings import settings

basic.load_theory('nat')
settings.unicode = True

## Predicate logic

In this section, we introduce rules for reasoning and major theorems in predicate logic. We have already encountered forall and exists quantifiers. Recall that they can be constructed as follows:

In [3]:
x = Var('x', NatType)
P = Var('P', TFun(NatType, BoolType))
Q = Var('Q', TFun(NatType, BoolType))
print(Forall(x, P(x)))
print(Exists(x, Q(x)))

∀x. P x
∃x. Q x


Higher-order logic defines forall quantification as a primitive operation, and introduction and elimination rules as primitive deduction rules. They are as follows:

$$ \frac{A \vdash t}{A \vdash \forall x.\,t} \hbox{ forall_intr ($x$ does not appear in $A$)}$$

This rule stipulates that if a proposition $t$, possibly depending on $x$, can be proved from assumptions not containing $x$, then the proposition $\forall x.\,t$ holds. This formalizes the general approach for proving a statement of the form $\forall x.\, P(x)$: we assume a new variable $x$ and show that $P(x)$ holds.

This rule can be invoked as follows:

In [4]:
th = Thm([], P(x))
print('th: ', th)
th2 = Thm.forall_intr(x, th)
print('th2:', th2)

th:  ⊢ P x
th2: ⊢ ∀x. P x


The elimination rule stipulates that if $\forall x.\,P(x)$ holds, then $P(s)$ holds for any term $s$ of the same type as $x$:

$$ \frac{\vdash \forall x.\, t}{\vdash t[s/x]} \hbox{ forall_elim} $$

This rule can be invoked as follows:

In [5]:
th3 = Thm.forall_elim(Nat(2), th2)
print('th3;', th3)

th3; ⊢ P 2


The interface for proof terms is similar, except we can call `forall_intr` and `forall_elim` directly from proof term objects:

In [6]:
pt = ProofTerm.assume(Forall(x, P(x)))
print('pt: ', pt)
pt2 = pt.forall_elim(Nat(2))
print('pt2:', pt2)
pt3 = apply_theorem('add_comm', inst=Inst(x=x, y=Nat(2)))
print('pt3:', pt3)
pt4 = pt3.forall_intr(x)
print('pt4:', pt4)

pt:  ProofTerm(∀x. P x ⊢ ∀x. P x)
pt2: ProofTerm(∀x. P x ⊢ P 2)
pt3: ProofTerm(⊢ x + 2 = 2 + x)
pt4: ProofTerm(⊢ ∀x::nat. x + 2 = 2 + x)


We now do a full example.

#### Example:

Prove $ (\forall x.\, P(x) \wedge Q(x)) \to (\forall x.\, P(x)) \wedge (\forall x.\, Q(x)) $.

#### Solution:

0. $\forall x.\, P(x)\wedge Q(x) \vdash \forall x.\, P(x)\wedge Q(x)$ by assume $\forall x.\, P(x)\wedge Q(x))$.
1. $\forall x.\, P(x)\wedge Q(x) \vdash P(x)\wedge Q(x)$ by forall\_elim $x$ from 0.
2. $\forall x.\, P(x)\wedge Q(x) \vdash P(x)$ by apply\_theorem conjD1 from 1.
3. $\forall x.\, P(x)\wedge Q(x) \vdash Q(x)$ by apply\_theorem conjD2 from 1.
4. $\forall x.\, P(x)\wedge Q(x) \vdash \forall x.\, P(x)$ by forall\_intr $x$ from 2.
5. $\forall x.\, P(x)\wedge Q(x) \vdash \forall x.\, Q(x)$ by forall\_intr $x$ from 3.
6. $\forall x.\, P(x)\wedge Q(x) \vdash (\forall x.\, P(x)) \wedge (\forall x.\, Q(x))$ by apply\_theorem conjI from 4, 5.
7. $\vdash (\forall x.\, P(x)\wedge Q(x)) \to (\forall x.\, P(x)) \wedge (\forall x.\, Q(x))$ by implies_\intr $\forall x.\, P(x)\wedge Q(x)$ from 6.

The Python code is as follows:

In [7]:
pt0 = ProofTerm.assume(Forall(x, And(P(x), Q(x))))
pt1 = pt0.forall_elim(x)
pt2 = apply_theorem('conjD1', pt1)
pt3 = apply_theorem('conjD2', pt1)
pt4 = pt2.forall_intr(x)
pt5 = pt3.forall_intr(x)
pt6 = apply_theorem('conjI', pt4, pt5)
pt7 = pt6.implies_intr(Forall(x, And(P(x), Q(x))))
print(pt7)

ProofTerm(⊢ (∀x. P x ∧ Q x) ⟶ (∀x. P x) ∧ (∀x. Q x))


One advantage of using proof terms is that we can let Python print out a human-readable proof

In [8]:
print(pt7.export())

0: ∀x. P x ∧ Q x ⊢ ∀x. P x ∧ Q x by assume ∀x. P x ∧ Q x
1: ∀x. P x ∧ Q x ⊢ P x ∧ Q x by forall_elim x from 0
2: ∀x. P x ∧ Q x ⊢ P x by apply_theorem conjD1 from 1
3: ∀x. P x ∧ Q x ⊢ ∀x. P x by forall_intr x from 2
4: ∀x. P x ∧ Q x ⊢ Q x by apply_theorem conjD2 from 1
5: ∀x. P x ∧ Q x ⊢ ∀x. Q x by forall_intr x from 4
6: ∀x. P x ∧ Q x ⊢ (∀x. P x) ∧ (∀x. Q x) by apply_theorem conjI from 3, 5
7: ⊢ (∀x. P x ∧ Q x) ⟶ (∀x. P x) ∧ (∀x. Q x) by implies_intr ∀x. P x ∧ Q x from 6


Note the order of the steps are slightly adjusted, as it depends on the details of the procedure for exporting proof terms.

## Exists quantification

The other quantifier in predicate logic is exists. In higher-order logic, this is not a primitive operation, and the introduction and elimination rules are given as theorems (just as for conjunction and disjunction). The theorems are:

In [9]:
print(theory.get_theorem('exI'))
print(theory.get_theorem('exE'))

⊢ ?P ?a ⟶ (∃a1. ?P a1)
⊢ (∃a. ?P a) ⟶ (∀a. ?P a ⟶ ?C) ⟶ ?C


The theorem `exI` is not difficult to understand: if $P(a)$ holds for some predicate $P$ and term $a$, then $\exists x.\,P(x)$ holds. The theorem `exE` is trickier and takes some time to get used to. Its use corresponds to the idea that if $\exists x\,P(x)$ holds, then we can take a fresh variable $a$ and assume $P(a)$. When applying `exE`, the variable $C$ is instantiated to the current goal, so the goal is changed from $C$ to $\forall a. P(a)\to C$. We then immediately apply `forall_elim` and `implies_elim` to get a new variable $a$ and assumption $P(a)$, and the conclusion is still $C$.

Let's demonstrate the use of `exI` and `exE` on an example.

#### Example:

Prove $(\exists x.\,P(x) \wedge Q(x)) \to (\exists x.\,P(x)) \wedge (\exists x.\,Q(x))$.

#### Solution:

0. $\exists x.\,P(x) \wedge Q(x) \vdash \exists x.\,P(x) \wedge Q(x)$ by assume $\exists x.\,P(x) \wedge Q(x)$.
1. $P(x) \wedge Q(x) \vdash P(x) \wedge Q(x)$ by assume $P(x) \wedge Q(x)$.
2. $P(x) \wedge Q(x) \vdash P(x)$ by apply_theorem conjD1 from 1.
3. $P(x) \wedge Q(x) \vdash \exists x.\,P(x)$ by apply_theorem exI from 2.
4. $P(x) \wedge Q(x) \vdash Q(x)$ by apply_theorem conjD2 from 1.
5. $P(x) \wedge Q(x) \vdash \exists x.\,Q(x)$ by apply_theorem exI from 4.
6. $P(x) \wedge Q(x) \vdash (\exists x.\,P(x)) \wedge (\exists x.\,Q(x))$ by apply_theorem conjI from 3, 5.
7. $\vdash P(x) \wedge Q(x) \to (\exists x.\,P(x)) \wedge (\exists x.\,Q(x))$ by implies_intr $P(x) \wedge Q(x)$ from 6.
8. $\vdash \forall x. P(x) \wedge Q(x) \to (\exists x.\,P(x)) \wedge (\exists x.\,Q(x))$ by forall_intr $x$ from 7.
9. $\exists x.\,P(x) \wedge Q(x) \vdash (\exists x.\,P(x)) \wedge (\exists x.\,Q(x))$ by apply_theorem exE from 0, 8.
10. $\vdash (\exists x. P(x) \wedge Q(x)) \to (\exists x.\,P(x)) \wedge (\exists x.\,Q(x))$ by implies_intr $\exists x.\,P(x) \wedge Q(x)$ from 9.

In this proof, steps 1 through 8 proves $(\exists x.\,P(x) \wedge (\exists x.\,Q(x))$ given a fresh variable $x$ and assumption $P(x)\wedge Q(x)$. The result of step 8 is then used in step 9 when applying `exE`.

The Python code is as follows:

In [10]:
pt0 = ProofTerm.assume(Exists(x, And(P(x), Q(x))))
pt1 = ProofTerm.assume(And(P(x), Q(x)))
pt2 = apply_theorem('conjD1', pt1)
pt3 = apply_theorem('conjD2', pt1)
pt4 = apply_theorem('exI', pt2, inst=Inst(P=P, a=x))
pt5 = apply_theorem('exI', pt3, inst=Inst(P=Q, a=x))
pt6 = apply_theorem('conjI', pt4, pt5)
pt7 = pt6.implies_intr(And(P(x), Q(x)))
pt8 = pt7.forall_intr(x)
pt9 = apply_theorem('exE', pt0, pt8)
pt10 = pt9.implies_intr(Exists(x, And(P(x), Q(x))))
print(pt10)

ProofTerm(⊢ (∃x. P x ∧ Q x) ⟶ (∃a1. P a1) ∧ (∃a1. Q a1))


Again, we can automatically print a human-readable proof:

In [11]:
print(pt10.export())

0: ∃x. P x ∧ Q x ⊢ ∃x. P x ∧ Q x by assume ∃x. P x ∧ Q x
1: P x ∧ Q x ⊢ P x ∧ Q x by assume P x ∧ Q x
2: P x ∧ Q x ⊢ P x by apply_theorem conjD1 from 1
3: P x ∧ Q x ⊢ ∃a1. P a1 by apply_theorem_for exI, {P: P, a: x} from 2
4: P x ∧ Q x ⊢ Q x by apply_theorem conjD2 from 1
5: P x ∧ Q x ⊢ ∃a1. Q a1 by apply_theorem_for exI, {P: Q, a: x} from 4
6: P x ∧ Q x ⊢ (∃a1. P a1) ∧ (∃a1. Q a1) by apply_theorem conjI from 3, 5
7: ⊢ P x ∧ Q x ⟶ (∃a1. P a1) ∧ (∃a1. Q a1) by implies_intr P x ∧ Q x from 6
8: ⊢ ∀x. P x ∧ Q x ⟶ (∃a1. P a1) ∧ (∃a1. Q a1) by forall_intr x from 7
9: ∃x. P x ∧ Q x ⊢ (∃a1. P a1) ∧ (∃a1. Q a1) by apply_theorem exE from 0, 8
10: ⊢ (∃x. P x ∧ Q x) ⟶ (∃a1. P a1) ∧ (∃a1. Q a1) by implies_intr ∃x. P x ∧ Q x from 9


As a third example, we prove a theorem with a mix of forall and exists quantifiers.

#### Example:

Prove $(\exists x. \forall y. R(x,y)) \to (\forall y. \exists x. R(x,y))$.

#### Solution:

0. $\exists x.\,\forall y.\,R(x,y) \vdash \exists x.\,\forall y.\,R(x,y)$ by assume $\exists x.\,\forall y.\,R(x,y)$.
1. $\forall y.\,R(x,y) \vdash \forall y.\,R(x,y)$ by assume $\forall y.\,R(x,y)$.
2. $\forall y.\,R(x,y) \vdash R(x,y)$ by forall_elim $y$ from 1.
3. $\forall y.\,R(x,y) \vdash \exists x.\,R(x,y)$ by apply_theorem exI from 2.
4. $\forall y.\,R(x,y) \vdash \forall y.\,\exists x.\,R(x,y)$ by forall_intr $y$ from 3.
5. $\vdash (\forall y. R(x,y)) \to (\forall y.\,\forall x.\,R(x,y))$ by implies_intr $\forall y.\,R(x,y)$ from 4.
6. $\vdash \forall x.\,(\forall y.\,R(x,y)) \to (\forall y.\,\exists x.\,R(x,y))$ by forall_intr $x$ from 5.
7. $\exists x.\,\forall y.\,R(x,y) \vdash \forall y.\,\exists x.\,R(x,y)$ by apply_theorem exE from 0, 6.
8. $\vdash (\exists x.\,\forall y.\,R(x,y)) \to (\forall y.\,\exists x.\,R(x,y))$ by implies_intr $\exists x.\,\forall y.\,R(x,y)$ from 7.

This is easily the trickiest proof to understand so far. Steps 1 through 6 establishes $\forall y.\,\exists x.\,R(x,y)$ for a fresh variable $x$ and assumption $\forall y.\,R(x,y)$. This is achieved by first assuming $\forall y.\,R(x,y)$ (step 1), then show $\exists x.\,R(x,y)$ for an arbitrary $y$ (step 2-3), from which it follows $\forall y.\,\exists x.\,R(x,y)$ (step 4). The result of step 6 is then used in step 7 when applying `exE`.

The Python code is as follows:

In [12]:
R = Var('R', TFun(NatType, NatType, BoolType))
x = Var('x', NatType)
y = Var('y', NatType)

pt0 = ProofTerm.assume(Exists(x, Forall(y, R(x,y))))
pt1 = ProofTerm.assume(Forall(y, R(x,y)))
pt2 = pt1.forall_elim(y)
pt3 = apply_theorem('exI', pt2, inst=Inst(P=Lambda(x, R(x,y))))
pt4 = pt3.forall_intr(y)
pt5 = pt4.implies_intr(Forall(y, R(x,y)))
pt6 = pt5.forall_intr(x)
pt7 = apply_theorem('exE', pt0, pt6)
pt8 = pt7.implies_intr(Exists(x, Forall(y, R(x,y))))
print(pt8)

ProofTerm(⊢ (∃x. ∀y. R x y) ⟶ (∀y. ∃a1. R a1 y))


In [13]:
print(pt8.export())

0: ∃x. ∀y. R x y ⊢ ∃x. ∀y. R x y by assume ∃x. ∀y. R x y
1: ∀y. R x y ⊢ ∀y. R x y by assume ∀y. R x y
2: ∀y. R x y ⊢ R x y by forall_elim y from 1
3: ∀y. R x y ⊢ ∃a1. R a1 y by apply_theorem_for exI, {P: λx. R x y, a: x} from 2
4: ∀y. R x y ⊢ ∀y. ∃a1. R a1 y by forall_intr y from 3
5: ⊢ (∀y. R x y) ⟶ (∀y. ∃a1. R a1 y) by implies_intr ∀y. R x y from 4
6: ⊢ ∀x. (∀y. R x y) ⟶ (∀y. ∃a1. R a1 y) by forall_intr x from 5
7: ∃x. ∀y. R x y ⊢ ∀y. ∃a1. R a1 y by apply_theorem exE from 0, 6
8: ⊢ (∃x. ∀y. R x y) ⟶ (∀y. ∃a1. R a1 y) by implies_intr ∃x. ∀y. R x y from 7


## Abstraction

The abstraction rule is the last of the primitive rules in higher-order logic. It deals with how to show the equality between two $\lambda$ terms.

$$ \frac{A \vdash t_1 = t_2}{A \vdash (\lambda x.\,t_1) = (\lambda x.\,t_2)} \hbox{ abstraction ($x$ does not appear in $A$)} $$

It is invoked as follows in Python. At the theorem level:

In [14]:
x = Var('x', NatType)
th = Thm([], Eq(x + Nat(2), Nat(2) + x))
print('th: ', th)
th2 = Thm.abstraction(x, th)
print('th2:', th2)

th:  ⊢ x + 2 = 2 + x
th2: ⊢ (λx::nat. x + 2) = (λx. 2 + x)


And at the proof term level:

In [15]:
pt = apply_theorem('add_comm', inst=Inst(x=x, y=Nat(2)))
print('pt: ', pt)
pt2 = pt.abstraction(x)
print('pt2:', pt2)

pt:  ProofTerm(⊢ x + 2 = 2 + x)
pt2: ProofTerm(⊢ (λx::nat. x + 2) = (λx. 2 + x))


We will show how to perform rewriting of abstractions at the conversion level in the next section.